In [1]:
#####
# This tutorial is from Devseed's Random Forest Model for Crop Type and LC
# http://devseed.com/sat-ml-training/Randomforest_cropmapping-with_GEE

In [40]:
from os import path as op
import pickle
import os

import geopandas as gpd
import pandas as pd
import shapely as shp
import matplotlib.pyplot as plt
import numpy as np
import rasterio as rio
from rasterio.features import rasterize
from rasterstats.io import bounds_window
import rasterstats
import folium

from shapely.geometry import Point, Polygon

from rasterio.plot import reshape_as_raster, reshape_as_image

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble._forest import ForestClassifier, ForestRegressor
import lime

In [44]:
labels_pine_df

,Unnamed: 0,parcel_id_year,volume_mean,iu30_mean,basal_mean,height_mean,dq_mean,basal_tot_mean,dbh_mean,species,inv_year,age_2019,species.1,lon,lat,geometry
0,414642,20110-538658-2016,420.940000,412.500000,59.540000,19.040000,28.380000,59.5,27.1,PINUS,2016,23,PINUS,-73.578048,-37.813762,POINT (-73.57805 -37.81376)
1,434810,20246-535806-2014,540.743750,528.747917,68.616667,22.635417,29.139583,68.6,27.4,PINUS,2014,25,PINUS,-73.570374,-37.821603,POINT (-73.57037 -37.82160)
2,434789,20246-492833-2004,81.375000,0.000000,22.658333,9.483333,16.675000,22.6,0.0,PINUS,2004,25,PINUS,-73.570706,-37.821013,POINT (-73.57071 -37.82101)
3,434805,20246-535806-2014,540.743750,528.747917,68.616667,22.635417,29.139583,68.6,27.4,PINUS,2014,25,PINUS,-73.571527,-37.820716,POINT (-73.57153 -37.82072)
4,434836,20246-535806-2014,540.743750,528.747917,68.616667,22.635417,29.139583,68.6,27.4,PINUS,2014,25,PINUS,-73.570409,-37.819801,POINT (-73.57041 -37.81980)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231788,814524,80003-537535-2016,68.980000,0.000000,20.050000,10.807500,13.627500,20.1,13.1,PINUS,2016,10,PINUS,-73.026291,-37.027302,POINT (-73.02629 -37.02730)
231789,815155,80003-542106-2017,141.607463,0.000000,28.789552,14.262687,16.174627,28.8,15.6,PINUS,2017,10,PINUS,-73.026445,-37.026995,POINT (-73.02644 -37.02699)
231790,814319,80003-531921-2014,23.376768,17.268687,12.346465,7.896970,10.580808,12.5,10.3,PINUS,2014,10,PINUS,-73.022386,-37.026953,POINT (-73.02239 -37.02695)
231791,815085,80003-542105-2017,130.437912,0.000000,26.651099,14.205495,16.036813,26.7,15.5,PINUS,2017,10,PINUS,-73.023507,-37.026946,POINT (-73.02351 -37.02695)


In [2]:
# training_vectors = gpd.read_file('../data/training/2019_arauco_lcTraining.shp')
# bbox = training_vectors.total_bounds
# center = shp.geometry.box(bbox[0], bbox[1], bbox[2], bbox[3]).centroid

# # show the 1st 5 lines
# training_vectors.head()

# # find all unique values of training data names to use as classes
# classes = np.unique(training_vectors['class'])
# classes

# # create a dictionary to convert class names into integers for modeling
# class_dict = dict(zip(classes, range(len(classes))))
# class_dict 

# raster_file = '/Users/brian.h.lee/Desktop/ch3/chile_forests/data/merged/2019_study_allrs10b.tif'

# # raster information

# ##If you want to read the data directly from the shared folder, uncomment the following line.
# # raster_file = '/content/drive/Shared drives/servir-sat-ml/data/Trans_nzoia_2019_05-02.tif'


# # a custom function for getting each value from the raster
# def all_values(x):
#     return x

# # this larger cell reads data from a raster file for each training vector
# X_raw = []
# y_raw = []
# with rasterio.open(raster_file, 'r') as src:
#     for (label, geom) in zip(training_vectors['class'], training_vectors['geometry']):
        
#         # read the raster data matching the geometry bounds
#         window = bounds_window(geom.bounds, src.transform)
#         # store our window information
#         window_affine = src.window_transform(window)
#         fsrc = src.read(window=window)
#         # rasterize the geometry into the larger shape and affine
#         mask = rasterize(
#             [(geom, 1)],
#             out_shape=fsrc.shape[1:],
#             transform=window_affine,
#             fill=0,
#             dtype='uint8',
#             all_touched=True
#         ).astype(bool)
        
#         # for each label pixel (places where the mask is true)
#         label_pixels = np.argwhere(mask)
        
#         for (row, col) in label_pixels:
#             # add a pixel of data to X
#             data = fsrc[:,row,col]
#             one_x = np.nan_to_num(data, nan=1e-3)
#             X_raw.append(one_x)
#             # add the label to y
#             y_raw.append(class_dict[label])
            
# # convert the training data lists into the appropriate numpy array shape and format for scikit-learn
# X = np.array(X_raw)
# y = np.array(y_raw)
# (X.shape, y.shape)

# # helper function for calculating ND*I indices (bands in the final dimension)
# def band_index(arr, a, b):
#     return np.expand_dims((arr[..., a] - arr[..., b]) / (arr[..., a] + arr[..., b]), axis=1)

# ndvi = band_index(X, 5, 4)
# # ndwi = band_index(X, 1, 3)

# X = np.concatenate([X, ndvi], axis=1)
# X.shape

# # split the data into test and train sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # calculate class weights to allow for training on inbalanced training samples
# labels, counts = np.unique(y_train, return_counts=True)
# class_weight_dict = dict(zip(labels, 1 / counts))
# class_weight_dict

# # initialize a RandomForestClassifier
# clf = RandomForestClassifier(
#     n_estimators=200,
#     class_weight=class_weight_dict,
#     max_depth=6,
#     n_jobs=-1,
#     verbose=1,
#     random_state=0)

# # fit the model to the data (training)
# clf.fit(X, y)

# # predict on X_test to evaluate the model
# preds = clf.predict(X_test)
# cm = confusion_matrix(y_test, preds, labels=labels)

# root_dir = '/Users/brian.h.lee/Desktop/ch3/chile_forests/models'

# # (optional) save the trained model as python pickle file
# model_name = op.join(root_dir,'random_forest_2019_lc.sav')
# with open(model_name, 'wb') as modelfile:
#     pickle.dump(clf, modelfile)
    
#     # plot the confusion matrix
# %matplotlib inline
# cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
# fig, ax = plt.subplots(figsize=(10, 10))
# im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
# ax.figure.colorbar(im, ax=ax)

# # We want to show all ticks...
# ax.set(xticks=np.arange(cm.shape[1]),
#        yticks=np.arange(cm.shape[0]),
#        # ... and label them with the respective list entries
#        xticklabels=classes, yticklabels=classes,
#        title='Normalized Confusion Matrix',
#        ylabel='True label',
#        xlabel='Predicted label')

# # Rotate the tick labels and set their alignment.
# plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
#          rotation_mode="anchor")

# # Loop over data dimensions and create text annotations.
# fmt = '.2f'
# thresh = cm.max() / 2.
# for i in range(cm.shape[0]):
#     for j in range(cm.shape[1]):
#         ax.text(j, i, format(cm[i, j], fmt),
#                 ha="center", va="center",
#                 color="white" if cm[i, j] > thresh else "black")
# fig.tight_layout()

# importances = clf.feature_importances_
# std = np.std([tree.feature_importances_ for tree in clf.estimators_], axis=0)

# band_names = ['ALOSHH', 'ALOSHHHV', 'ALOSHV', 'S2BLUE', 'S2GREEN', 'S2RED', 'S1VH', 'S1VV', 'S1VVVH', 'S2NIR', 'NDVI']

# forest_importances = pd.Series(importances, index=band_names)

# fig, ax = plt.subplots()
# forest_importances.plot.bar(yerr=std, ax=ax)
# ax.set_title("Feature importances using MDI")
# ax.set_ylabel("Mean decrease in impurity")
# fig.tight_layout()

# # predict again with the tree interpreter to see how much each band contributes to the classification
# sample = 100
# prediction, bias, contributions = ti.predict(clf, X_test[:sample])
# c = np.sum(contributions, axis=0)

# # plot the contributions
# band_names = ['ALOSHH', 'ALOSHHHV', 'ALOSHV', 'S2BLUE', 'S2GREEN', 'S2RED', 'S1VH', 'S1VV', 'S1VVVH', 'S2NIR', 'NDVI']

# gdf = gpd.GeoDataFrame(c, columns=classes, index=band_names)
# gdf.style.background_gradient(cmap='viridis')

# # in this case, we predict over the entire input image
# # (only small portions were used for training)
# new_image = raster_file

# # specify the output
# output_image = op.join(root_dir, "classification.tif")

# with rasterio.open(new_image, 'r') as src:
#     profile = src.profile
#     profile.update(
#         dtype=rasterio.uint8,
#         count=1,
#     )

#     with rasterio.open(output_image, 'w', **profile) as dst:

        
#         # perform prediction on each small image patch to minimize required memory
#         patch_size = 500

#         for i in range((src.shape[0] // patch_size) + 1):
#             for j in range((src.shape[1] // patch_size) + 1):
#                 # define the pixels to read (and write) with rasterio windows reading
#                 window = rasterio.windows.Window(
#                     j * patch_size,
#                     i * patch_size,
#                     # don't read past the image bounds
#                     min(patch_size, src.shape[1] - j * patch_size),
#                     min(patch_size, src.shape[0] - i * patch_size))
                
#                 # read the image into the proper format
#                 data = src.read(window=window)
                
#                 # adding indices if necessary
#                 img_swp = np.moveaxis(data, 0, 2)
#                 img_flat = img_swp.reshape(-1, img_swp.shape[-1])

#                 img_ndvi = band_index(img_flat, 5, 4)
#                 img_ndwi = band_index(img_flat, 1, 3)

#                 img_w_ind = np.concatenate([img_flat, img_ndvi, img_ndwi], axis=1)

#                 # remove no data values, store the indices for later use
#                 m = np.ma.masked_invalid(img_w_ind)
#                 to_predict = img_w_ind[~m.mask].reshape(-1, img_w_ind.shape[-1])

#                 # skip empty inputs
#                 if not len(to_predict):
#                     continue
#                 # predict
#                 img_preds = clf.predict(to_predict)

#                 # add the prediction back to the valid pixels (using only the first band of the mask to decide on validity)
#                 # makes the assumption that all bands have identical no-data value arrangements
#                 output = np.zeros(img_flat.shape[0])
#                 output[~m.mask[:, 0]] = img_preds.flatten()
#                 # resize to the original image dimensions
#                 output = output.reshape(*img_swp.shape[:-1])

#                 # create our final mask
#                 mask = (~m.mask[:, 0]).reshape(*img_swp.shape[:-1])

#                 # write to the final files
#                 dst.write(output.astype(rasterio.uint8), 1, window=window)
#                 dst.write_mask(mask, window=window)



In [3]:
# Load pretrained model
os.getcwd()
os.chdir('../models/rf/')
loaded_model = pickle.load(open('random_forest_2019_lc.sav', 'rb'))

/Users/brianlee/opt/anaconda3/envs/chile_forest/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.1.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/brianlee/opt/anaconda3/envs/chile_forest/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.1.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
image_directory = '/Users/brianlee/Desktop/ch3/chile_forests/data/merged/samples/input'
output_directory = '/Users/brianlee/Desktop/ch3/chile_forests/data/merged/samples/output'

In [5]:
os.chdir(image_directory)

In [27]:
# specify the output
output_image = op.join(output_directory, new_image)

In [28]:
def band_index(arr, a, b):
    return np.expand_dims((arr[..., a] - arr[..., b]) / (arr[..., a] + arr[..., b]), axis=1)

In [35]:

for filename in os.listdir(image_directory):
    if filename.endswith ('.tif'):
        try:
            new_image = filename

            # specify the output
            output_image = op.join(output_directory, new_image)

            with rio.open(new_image, 'r') as src:
                profile = src.profile
                profile.update(
                    dtype=rio.uint8,
                    count=1,
                )

                with rio.open(output_image, 'w', **profile) as dst:


            #         # perform prediction on each small image patch to minimize required memory
            #         patch_size = 500

            #         for i in range((src.shape[0] // patch_size) + 1):
            #             for j in range((src.shape[1] // patch_size) + 1):
            #                 # define the pixels to read (and write) with rasterio windows reading
            #                 window = rio.windows.Window(
            #                     j * patch_size,
            #                     i * patch_size,
            #                     # don't read past the image bounds
            #                     min(patch_size, src.shape[1] - j * patch_size),
            #                     min(patch_size, src.shape[0] - i * patch_size))

                            # read the image into the proper format
                            data = src.read()

                            new_data = np.nan_to_num(data, copy=True, nan=0.0, posinf=None, neginf=None)

                            # new_data = to_numpy(df)

                            # adding indices if necessary
                            img_swp = np.moveaxis(new_data, 0, 2)
                            img_flat = img_swp.reshape(-1, img_swp.shape[-1])

                            img_ndvi = band_index(img_flat, 9, 6)
                            img_ndvi = np.nan_to_num(img_ndvi, copy=True, nan=0.0, posinf=None, neginf=None)

                            # img_ndwi = band_index(img_flat, 9, 5)

                            img_w_ind = np.concatenate([img_flat, img_ndvi], axis=1)

                            # remove no data values, store the indices for later use
                            # m = np.ma.masked_invalid(img_w_ind)
                            to_predict = img_w_ind.reshape(-1, img_w_ind.shape[-1])

                            # # skip empty inputs
                            # if not len(to_predict):
                            #     continue
                            # predict
                            img_preds = loaded_model.predict(to_predict)

                            # add the prediction back to the valid pixels (using only the first band of the mask to decide on validity)
                            # makes the assumption that all bands have identical no-data value arrangements
                            output = np.zeros(img_flat.shape[0])
                            output = img_preds.flatten()
                            # resize to the original image dimensions
                            output = output.reshape(*img_swp.shape[:-1])

                            # create our final mask
                            # mask = (~m.mask[:, 0]).reshape(*img_swp.shape[:-1])

                            # write to the final files
                            dst.write(output.astype(rio.uint8), 1)
                            # dst.write_mask(mask, window=window)
                            
        except:
            print(new_image)
            pass

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    0.2s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:    0.8s
[Parallel(n_jobs=10)]: Done 200 out of 200 | elapsed:    0.9s finished
/var/folders/dx/35d3rjys675ftkqgrd2j85lc0000gn/T/ipykernel_62005/3126048752.py:2: RuntimeWarning: invalid value encountered in true_divide
  return np.expand_dims((arr[..., a] - arr[..., b]) / (arr[..., a] + arr[..., b]), axis=1)
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    0.7s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:    3.4s
[Parallel(n_jobs=10)]: Done 200 out of 200 | elapsed:    3.7s finished
/var/folders/dx/35d3rjys675ftkqgrd2j85lc0000gn/T/ipykernel_62005/3126048752.py:2: RuntimeWarning: invalid value encountered in true_divide
  return np.expand_dims((arr[..., a] - arr[..., b]) / (arr[..., a] + arr[...,